# Here's a simple implementation of bilinear interpolation on tensors using PyTorch.

I wrote this up since I ended up learning a lot about options for interpolation in both the numpy and PyTorch ecosystems. More generally than just interpolation, too, it's also a nice case study in how PyTorch magically can put very numpy-like code on the GPU (and by the way, do autodiff for you too).

For interpolation in PyTorch, this open issue calls for more interpolation features. There is now a nn.functional.grid_sample() feature but at least at first this didn't look like what I needed (but we'll come back to this later).

In particular I wanted to take an image, W x H x C, and sample it many times at different random locations. Note also that this is different than upsampling which exhaustively samples and also doesn't give us flexibility with the precision of sampling.

In [1]:
import numpy as np

def bilinear_interpolate_numpy(im, x, y):
    x0 = np.floor(x).astype(int)
    x1 = x0 + 1
    y0 = np.floor(y).astype(int)
    y1 = y0 + 1

    x0 = np.clip(x0, 0, im.shape[1]-1)
    x1 = np.clip(x1, 0, im.shape[1]-1)
    y0 = np.clip(y0, 0, im.shape[0]-1)
    y1 = np.clip(y1, 0, im.shape[0]-1)

    Ia = im[ y0, x0 ]
    Ib = im[ y1, x0 ]
    Ic = im[ y0, x1 ]
    Id = im[ y1, x1 ]

    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)

    return (Ia.T*wa).T + (Ib.T*wb).T + (Ic.T*wc).T + (Id.T*wd).T

In [2]:
import torch
dtype = torch.cuda.FloatTensor
dtype_long = torch.cuda.LongTensor

def bilinear_interpolate_torch(im, x, y):
    x0 = torch.floor(x).type(dtype_long)
    x1 = x0 + 1
    
    y0 = torch.floor(y).type(dtype_long)
    y1 = y0 + 1

    x0 = torch.clamp(x0, 0, im.shape[1]-1)
    x1 = torch.clamp(x1, 0, im.shape[1]-1)
    y0 = torch.clamp(y0, 0, im.shape[0]-1)
    y1 = torch.clamp(y1, 0, im.shape[0]-1)
    
    Ia = im[ y0, x0 ][0]
    Ib = im[ y1, x0 ][0]
    Ic = im[ y0, x1 ][0]
    Id = im[ y1, x1 ][0]
    
    wa = (x1.type(dtype)-x) * (y1.type(dtype)-y)
    wb = (x1.type(dtype)-x) * (y-y0.type(dtype))
    wc = (x-x0.type(dtype)) * (y1.type(dtype)-y)
    wd = (x-x0.type(dtype)) * (y-y0.type(dtype))

    return torch.t((torch.t(Ia)*wa)) + torch.t(torch.t(Ib)*wb) + torch.t(torch.t(Ic)*wc) + torch.t(torch.t(Id)*wd)

# Testing for correctness
Bilinear interpolation is very simple but there are a few things that can be easily messed up.

I did a quick comparison for correctness with SciPy's interp2d.

Side note: there are actually a ton of interpolation options in SciPy but none I tested met my critera of (a) doing bilinear interpolation for high-dimensional spaces and (b) efficiently use gridded data. The ones I tested that were built for many dimensions were requiring me to specify sample points for all of those dimensions (and doing trilinear, or other) interpolation. I could get LinearNDInterpolator to do bilinear interpolation for high dimensional vectors but this does not meet criteria (b). There's probably a better option but, at any rate, I gave up and went back to my numpy and PyTorch options.


In [9]:
# Also use scipy to check for correctness
import scipy.interpolate
def bilinear_interpolate_scipy(image, x, y):
    x_indices = np.arange(image.shape[0])
    y_indices = np.arange(image.shape[1])
    interp_func = scipy.interpolate.interp2d(x_indices, y_indices, image, kind='linear')
    return interp_func(x,y)

# Make small sample data that's easy to interpret
image = np.ones((5,5))
image[3,3] = 4
image[3,4] = 3

sample_x, sample_y = np.asarray([3.2]), np.asarray([3.4])

print ("numpy result:", bilinear_interpolate_numpy(image, sample_x, sample_y))
print ("scipy result:", bilinear_interpolate_scipy(image, sample_x, sample_y))

image = torch.unsqueeze(torch.FloatTensor(image).type(dtype),2)
sample_x = torch.FloatTensor([sample_x]).type(dtype)
sample_y = torch.FloatTensor([sample_y]).type(dtype)

print("torch result:{}".format(bilinear_interpolate_torch(image, sample_x, sample_y)))

numpy result: [2.68]
scipy result: [2.68]
torch result:
 2.6800
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]



# High dimensional bilinear interpolation
For the correctness test comparing with scipy, we couldn't do W x H x C interpolation for anything but C=1. Now though, we can do bilinear interpolation in either numpy or torch for arbitrary C:

In [12]:
# Do high dimensional bilinear interpolation in numpy and PyTorch
W, H, C = 25, 25, 7
image = np.random.randn(W, H, C)

num_samples = 4
samples_x, samples_y = np.random.rand(num_samples)*(W-1), np.random.rand(num_samples)*(H-1)

print (bilinear_interpolate_numpy(image, samples_x, samples_y))

image = torch.from_numpy(image).type(dtype)
samples_x = torch.FloatTensor([samples_x]).type(dtype)
samples_y = torch.FloatTensor([samples_y]).type(dtype)

print(bilinear_interpolate_torch(image, samples_x, samples_y))

[[-0.03487975  0.05992728 -0.32450413  1.73506905 -0.12563051  0.17523377
   0.66694222]
 [ 0.06435581  0.9336627  -0.31199044  0.33880198 -0.20322331 -0.31408149
   1.20000908]
 [ 0.21110027 -1.04296268 -0.58333472 -0.44477671 -0.39599681  0.98248355
  -0.41903184]
 [-0.23086448  0.44345576 -1.59667113  0.90401681  0.06551464 -0.75609012
   0.28220755]]

-0.0349  0.0599 -0.3245  1.7351 -0.1256  0.1752  0.6669
 0.0644  0.9337 -0.3120  0.3388 -0.2032 -0.3141  1.2000
 0.2111 -1.0430 -0.5833 -0.4448 -0.3960  0.9825 -0.4190
-0.2309  0.4435 -1.5967  0.9040  0.0655 -0.7561  0.2822
[torch.cuda.FloatTensor of size 4x7 (GPU 0)]



# Bechmarking: numpy (CPU) vs. pytorch (CPU) vs. pytorch (GPU)

In [13]:
# Timing comparison for WxHxC (where C is large for a high dimensional descriptor)
W, H, C = 640, 480, 32
image = np.random.randn(W, H, C)

num_samples = 10000
samples_x, samples_y = np.random.rand(num_samples)*(W-1), np.random.rand(num_samples)*(H-1)

import time

start = time.time()
bilinear_interpolate_numpy(image, samples_x, samples_y)
print ("numpy took       ", time.time() - start)

dtype = torch.FloatTensor
dtype_long = torch.LongTensor
image = torch.FloatTensor(image).type(dtype)
samples_x = torch.FloatTensor([samples_x]).type(dtype)
samples_y = torch.FloatTensor([samples_y]).type(dtype)

start = time.time()
bilinear_interpolate_torch(image, samples_x, samples_y)
print ("torch on CPU took", time.time() - start) 

dtype = torch.cuda.FloatTensor
dtype_long = torch.cuda.LongTensor
image = image.type(dtype)
samples_x = samples_x.type(dtype)
samples_y = samples_y.type(dtype)

start = time.time()
bilinear_interpolate_torch(image, samples_x, samples_y)
print ("torch on GPU took", time.time() - start)

numpy took        0.01631951332092285
torch on CPU took 0.20528769493103027
torch on GPU took 0.04155755043029785


# Using the available nn.functional.grid_sample()

In [15]:
import torch.nn.functional
dtype = torch.cuda.FloatTensor
dtype_long = torch.cuda.LongTensor

def bilinear_interpolate_torch_gridsample(image, samples_x, samples_y):
                                                # input image is: W x H x C
    image = image.permute(2,0,1)                # change to:      C x W x H
    image = image.unsqueeze(0)                  # change to:  1 x C x W x H
    samples_x = samples_x.unsqueeze(2)
    samples_x = samples_x.unsqueeze(3)
    samples_y = samples_y.unsqueeze(2)
    samples_y = samples_y.unsqueeze(3)
    samples = torch.cat([samples_x, samples_y],3)
    samples[:,:,:,0] = (samples[:,:,:,0]/(W-1)) # normalize to between  0 and 1
    samples[:,:,:,1] = (samples[:,:,:,1]/(H-1)) # normalize to between  0 and 1
    samples = samples*2-1                       # normalize to between -1 and 1
    return torch.nn.functional.grid_sample(image, samples)

# Correctness test
W, H, C = 5, 5, 1
test_image = torch.ones(W,H,C).type(dtype)
test_image[3,3,:] = 4
test_image[3,4,:] = 3

test_samples_x = torch.FloatTensor([[3.2]]).type(dtype)
test_samples_y = torch.FloatTensor([[3.4]]).type(dtype)

print (bilinear_interpolate_torch_gridsample(test_image, test_samples_x, test_samples_y))

# Benchmark
start = time.time()
bilinear_interpolate_torch_gridsample(image, samples_x, samples_y)
print ("torch gridsample took ", time.time() - start)

Variable containing:
(0 ,0 ,.,.) = 
  2.6800
[torch.cuda.FloatTensor of size 1x1x1x1 (GPU 0)]

torch gridsample took  0.0016460418701171875


# High dimensional bilinear interpolation
For the correctness test comparing with scipy, we couldn't do H x C interpolation for anything but C=1. Now though, we can do bilinear interpolation in either numpy or torch for arbitrary C:

In [17]:
# Do high dimensional bilinear interpolation in numpy and PyTorch
W, H, C = 25, 25, 7
image = np.random.randn(W, H, C)
print(image)

num_samples = 4
samples_x, samples_y = np.random.rand(num_samples)*(W-1), np.random.rand(num_samples)*(H-1)
print(samples_x)
print(samples_y)

print (bilinear_interpolate_numpy(image, samples_x, samples_y))

image = torch.from_numpy(image).type(dtype)
samples_x = torch.FloatTensor([samples_x]).type(dtype)
samples_y = torch.FloatTensor([samples_y]).type(dtype)

print(bilinear_interpolate_torch(image, samples_x, samples_y))

[[[ 0.80408495 -0.3605468  -0.49353077 ...  1.03704021 -0.72125335
    0.769743  ]
  [-0.37596455  0.38516289  1.8144742  ...  1.11418533  0.9892282
   -0.82162355]
  [ 1.06390295 -1.15098164  0.44397855 ... -0.34701702 -0.40195921
   -0.50611539]
  ...
  [-2.16148843 -0.28735546 -0.65934313 ... -2.49980777 -0.6550439
    0.31174935]
  [-1.26001232 -0.29804738  1.00841776 ... -0.07920075 -0.67197759
   -1.68075489]
  [ 0.47754207 -1.29339343 -0.15063431 ... -0.41592414  1.81205171
   -1.18107249]]

 [[ 0.66450915 -0.32658933  0.402464   ... -0.03107425 -0.99818075
   -0.25391999]
  [-0.87278078 -1.41842194  1.26444189 ... -0.78948578  0.70247765
    1.77515542]
  [-0.25086732  0.56394069  0.83971729 ... -0.68959276 -0.01290295
   -0.44932601]
  ...
  [-0.70048765 -0.63672346 -2.2091686  ...  1.14704603 -1.1555942
    0.86892644]
  [-0.08803624 -0.55184275 -0.96963481 ...  1.1300187  -0.30606453
   -0.44322507]
  [ 0.95104782 -0.43349358 -0.03476632 ...  0.7565349   1.69916694
   -0.808

In [18]:
import torch
import torch.nn as nn

def manual_bilinear(x1, x2, A, b):
    return torch.mm(x1, torch.mm(A, x2)) + b

x_ones = torch.ones(2)
x_zeros = torch.zeros(2)

# ---------------------------
# With Bias:

B = nn.Bilinear(2, 2, 1)
A = B.weight
print(B.bias)
# > tensor([-0.6748], requires_grad=True)
b = B.bias

print(B(x_ones, x_zeros))
# > tensor([-0.6748], grad_fn=<ThAddBackward>)
print(manual_bilinear(x_ones.view(1, 2), x_zeros.view(2, 1), A.squeeze(), b))
# > tensor([[-0.6748]], grad_fn=<ThAddBackward>)

print(B(x_ones, x_ones))
# > tensor([-1.7684], grad_fn=<ThAddBackward>)
print(manual_bilinear(x_ones.view(1, 2), x_ones.view(2, 1), A.squeeze(), b))
# > tensor([[-1.7684]], grad_fn=<ThAddBackward>)

Parameter containing:
 0.6416
[torch.FloatTensor of size 1]



RuntimeError: matrices expected, got 1D, 2D tensors at /pytorch/torch/lib/TH/generic/THTensorMath.c:1429

In [25]:
# ---------------------------
# Without Bias:
import torch
import torch.nn as nn

def manual_bilinear(x1, x2, A, b):
    return torch.mm(x1, torch.mm(A, x2)) + b

x_ones = torch.ones(2)
x_zeros = torch.zeros(2)
print(x_ones)
print(x_zeros)

B = nn.Bilinear(2, 2, 1, bias=False)
b = torch.zeros(1)

print(B(x_ones, x_zeros))
print(manual_bilinear(x_ones.view(1, 2), x_zeros.view(2, 1), A.squeeze(), b))

print(B(x_ones, x_ones))
print(manual_bilinear(x_ones.view(1, 2), x_ones.view(2, 1), A.squeeze(), b))


 1
 1
[torch.FloatTensor of size 2]


 0
 0
[torch.FloatTensor of size 2]



RuntimeError: matrices expected, got 1D, 2D tensors at /pytorch/torch/lib/TH/generic/THTensorMath.c:1429

In [27]:
import torch
import torch.nn as nn

m = nn.Bilinear(256, 256, 256)
input1 = torch.randn(128, 256)
input2 = torch.randn(128, 256)
output = m(input1, input2)
print(output.size())


RuntimeError: save_for_backward can only save input or output tensors, but argument 0 doesn't satisfy this condition